# Modeller etap 1 - szukanie struktur spokrewnionych z badaną sekwncją, w bazach danych


Przeszukiwanie bazy danych. Plik basic-example/pdb_95.pir zawiera pdb_id:łańczuch wraz z przypisanymi do nich sekwnecjami w formacie PIR. Bazę przeszukujemy względem sekwencji zapisanej w pliku "TvLDH.ali" (nie jest to nasza sekwencja z CASP15). Baza ta zawiera 11079 rekordów.

In [1]:
import numpy as np
from modeller import *

log.verbose()
env = Environ()

#-- Prepare the input files

#-- Read in the sequence database
sdb = SequenceDB(env)
sdb.read(seq_database_file='basic-example/pdb_95.pir', seq_database_format='PIR',
         chains_list='ALL', minmax_db_seq_len=(30, 4000), clean_sequences=True)

#-- Write the sequence database in binary form
sdb.write(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
          chains_list='ALL')


#-- Now, read in the binary database
sdb.read(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
         chains_list='ALL')


#-- Read in the target sequence/alignment
aln = Alignment(env)
aln.append(file='TvLDH.ali', alignment_format='PIR', align_codes='ALL')

#-- Convert the input sequence/alignment into
#   profile format
prf = aln.to_profile()

#-- Scan sequence database to pick up homologous sequences
prf.build(sdb, matrix_offset=-450, rr_file='${LIB}/blosum62.sim.mat',
          gap_penalties_1d=(-500, -50), n_prof_iterations=1,
          check_profile=False, max_aln_evalue=0.01)

#-- Write out the profile in text format
prf.write(file='build_profile.prf', profile_format='TEXT')

#-- Convert the profile back to alignment format
aln = prf.to_alignment()


#-- Write out the alignment file
aln.write(file='build_profile.ali', alignment_format='PIR')

ModuleNotFoundError: No module named 'modeller'

W Wyniku przeszkuwiania bazy danych znaleziono 29 sekwncji podobnych do sekwencji białka celu. Prosze zobaczyć plik wynikowy: "build_profile.prf".

# Etap 2 - wybieranie struktur które posłuża jako szablon w modelowaniu homologicznym

Wybeiramy 6 sekwncji charakteryzujace sie miarą istotności wyniku (E-value) równą  0.

In [2]:
aln = Alignment(env)

for (pdb, chain) in (('1b8p', 'A'), ('1bdm', 'A'), ('1civ', 'A'),
                     ('5mdh', 'A'), ('7mdh', 'A'), ('1smk', 'A')):
    m = Model(env, file='basic-example/'+pdb+'.pdb', model_segment=('FIRST:'+chain, 'LAST:'+chain))
    aln.append_model(m, atom_files='basic-example/'+pdb+'.pdb', align_codes=pdb+chain)
aln.malign()
aln.malign3d()
aln.compare_structures()
aln.id_table(matrix_file='family.mat')
env.dendrogram(matrix_file='family.mat', cluster_cut=-1.0)

Widzimy tutaj 3 grupy struktur które są strukturalnie do siebie podbne:
-5mdh:A, 1bdm:A, oraz 1b8p:A -> sposród tej grupy 1bdm ma najlepszy R-factor rozdzielczoc 
            oraz procent identycznoscioraz rozdzielczosc struktury
-1smk ma tylko 34% identyczności względem badanej sekwencji
-1civA oraz 7mdhA posiada gorszą rozdzielczosc

# Etap 3 - dopasowanie sekwencji szablonowych do badanej

In [5]:
env = Environ()
aln = Alignment(env)
mdl = Model(env, file='basic-example/1bdm', model_segment=('FIRST:A','LAST:A'))
aln.append_model(mdl, align_codes='1bdmA', atom_files='1bdm.pdb')
aln.append(file='TvLDH.ali', align_codes='TvLDH')
aln.align2d(max_gap_length=50)
aln.write(file='TvLDH-1bdmA.ali', alignment_format='PIR')
aln.write(file='TvLDH-1bdmA.pap', alignment_format='PAP')

Tutaj dokonujemy dopasowania sekwencji 1bdm:A względem badanej sekwencji(TvLDH). 
Prosze obejrzec plik TvLDH-1bdmA.ali

# Etap 4 - budowanie modelu

In [7]:
from modeller import *
from modeller.automodel import *
#from modeller import soap_protein_od

env = Environ()
aln = Alignment(env)
a = AutoModel(env, alnfile='TvLDH-1bdmA.ali',
              knowns='1bdmA', sequence='TvLDH',
              assess_methods=(assess.DOPE,
                              #soap_protein_od.Scorer(),
                              assess.GA341))
a.starting_model = 1
a.ending_model = 5
a.make()

Tutaj dokonujemy predykcji przez modelowanie homologiczne na podstawie dopasowania sekwencji 
w pliku TvLDH-1bdmA.ali 
oraz struktury zapisanej w pliku "1bdm.pdb"

# Etap 5 - Walidacja

In [10]:
from modeller import *
from modeller.scripts import complete_pdb

log.verbose()    # request verbose output
env = Environ()
env.libs.topology.read(file='$(LIB)/top_heav.lib') # read topology
env.libs.parameters.read(file='$(LIB)/par.lib') # read parameters

# read model file
mdl = complete_pdb(env, 'TvLDH.B99990002.pdb')

# Assess with DOPE:
s = Selection(mdl)   # all atom selection
s.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='TvLDH.profile',
              normalize_profile=True, smoothing_window=15)

In [29]:
from modeller import *
import os, sys

env = environ()
env.libs.topology.read(file='$(LIB)/top_heav.lib')
env.libs.parameters.read(file='$(LIB)/par.lib')

env.io.atom_files_directory = ['basic-example']
                               
mdl = model(env, file='1bdm', model_segment=('FIRST:A','LAST:A'))
s = selection(mdl)                             
s.assess_dope(output='ENERGY_PROFILE NO_REPORT',
                 file='1bdmA.profile',
                 normalize_profile=True,
                 smoothing_window=15)                            

In [32]:
from matplotlib import pyplot as plt

data_model=np.loadtxt('TvLDH.profile',comments=['#'],skiprows=1,usecols=(0, 41))
data_template=np.loadtxt('1bdmA.profile',comments=['#'],skiprows=1,usecols=(0, 41))

plt.plot(data_model[:,0],data_model[:,1],label='model')
plt.plot(data_template[:,0],data_template[:,1],label='template')
plt.legend()
plt.xlabel("Aligment position")
plt.ylabel("DOPE per-residue score")

Istnieje wiele metod oceny jakości modelu, które dostarczają wyników wspomagających proces jego udokładniania dzięki identyfikowaniu niepoprawnie przewidzianych fragmentów struktury przestrzennej białka celu. W tej części ćwiczenia zostanie wykorzystana metoda DOPE (ang. Discrete Optimized Protein Energy) zaimplemetowana w programie MODELLLER.

W przypadku kiedy wielkość potencjału DOPE dla szablonu jest zbliżona (różnica mniejsza niż 5%) do tej uzyskanej dla modelu, można zwykle przyjąć, że uzyskano dobry model białka.